## XOOM Scraper (July 2020)

### Best practice is to run each chunk by itself

In [1]:
# If you are not sure about having requests or beautifulsoup4 installed, go ahead to unquote and run this

'''
import sys

!{sys.executable} -m pip install requests

!{sys.executable} -m pip install bs4
'''

'\nimport sys\n\n!{sys.executable} -m pip install requests\n\n!{sys.executable} -m pip install bs4\n'

In [2]:
# Load library 

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
from datetime import datetime

### User inputs these 4 pieces of information

In [3]:
'''
You can visit https://docs.google.com/spreadsheets/d/1DaR63CcRQ-6XIo-q2mTpOjZ-I51dXdK8-uomusGtnuU/edit?usp=sharing
for reference of alpha2 country code
'''

sourceCountryCode = 'US'     
sourceCurrencyCode = 'USD'
destinationCountryCode = 'MX'
destinationCurrencyCode = 'MXN'

In [4]:
# Separated URLs to run the loop

url_1 = "https://www.xoom.com/calculate-fee-table?sourceCountryCode="
url_2 = "&sourceCurrencyCode="
url_3 = "&destinationCountryCode="
url_4 = "&destinationCurrencyCode="
url_5 = "&sendAmount="

In [5]:
# If you need to add more transaction sizes for your corridor, make sure to input as string object

txn_sizes = ["50","100","200","300","400","500","600","700","800","900","1000",
                      "1100","1200","1300","1400","1500","1600","1700","1800","1900","2000",
                      "2100","2200","2300","2400","2500","2600","2700","2800","2900","3000"]

# In the event that you want to run faster, we suggest to use this alternative size range
# To do that, go on to the next code box and change the section: 
'''
txn = txn_sizes to txn = txn_sizes_alt 
'''

txn_sizes_alt = ["50","100","200","300","400","500","600","700","800","900","1000",
                      "1100","1500","2000","2500","3000"]

In [6]:
# This is where to change the size scale

txn = txn_sizes

In [7]:
# Create list of the complete urls that we will be going through

list_urls = []

for s in txn:
    list_urls.append(str(url_1+sourceCountryCode+url_2+sourceCurrencyCode+url_3
                         +destinationCountryCode+url_4+destinationCurrencyCode+url_5+s))

In [8]:
# From XOOM, we are getting the following combinations:

'''
Xoom has deposit or pickup options, but
also tag 'summarized' for corridor that only has one deliver option
'''

payment_type = [' Bank account', ' Debit card', ' Credit card']

delivery_type = ['deposit','pickup','summarized']



In [9]:
# Making initial columns of the dataframe
full_list = []
send_country = []
receive_country = []
send_currency = []
receive_currency = []

for i in delivery_type:
    for j in payment_type:
        full_list.append(i+j)
        send_country.append(sourceCountryCode)
        send_currency.append(sourceCurrencyCode)
        receive_country.append(destinationCountryCode)
        receive_currency.append(destinationCurrencyCode)

In [10]:
# Checking the list of deliver and payment options
full_list

['deposit Bank account',
 'deposit Debit card',
 'deposit Credit card',
 'pickup Bank account',
 'pickup Debit card',
 'pickup Credit card',
 'summarized Bank account',
 'summarized Debit card',
 'summarized Credit card']

In [11]:
# Creating dataframe to store output

df = pd.DataFrame({'send_country': send_country, 'send_currency':send_currency, 
                  'receive_country':receive_country, 'receive_currency':receive_currency,
                  'payment_type': full_list})

In [12]:
df

,send_country,send_currency,receive_country,receive_currency,payment_type
0,US,USD,MX,MXN,deposit Bank account
1,US,USD,MX,MXN,deposit Debit card
2,US,USD,MX,MXN,deposit Credit card
3,US,USD,MX,MXN,pickup Bank account
4,US,USD,MX,MXN,pickup Debit card
5,US,USD,MX,MXN,pickup Credit card
6,US,USD,MX,MXN,summarized Bank account
7,US,USD,MX,MXN,summarized Debit card
8,US,USD,MX,MXN,summarized Credit card


## Now let's scrap some fees

In [13]:
# Create list of html pages request
# This will take a bit of time (about 5-7 minutes) since we want to put sleep time between run to avoid being flagged
# Can try to make sleep(7) to run faster but you might not get full information 

begin = datetime.today()

pages= []
for page in range(len(list_urls)):
    r = requests.get(list_urls[page])
    c = r.content
    soup = BeautifulSoup(c,"html.parser")
    pages.append(soup)
    time.sleep(10)

# print out the result time to see how long it takes
end = datetime.today()
print('runtime:', end - begin)

runtime: 0:05:34.705576


In [14]:
# Checking on the result
# Scroll to the end of the page to see if our request got flagged and if the corridor has send limit at 2,999
pages

[
 <div>
 <data id="jsonData">
             {"data":{"receiveAmount":"1104.00","fxRate":"22.0814","amountRounded":false,"currencyDisclaimer":"In addition to the transaction fee, Xoom also makes money when it changes your send currency into a different currency. Xoom rounds to the nearest whole peso.","sendAmount":"50.00","remittanceResourceID":"8584d437-b959-486d-906a-33f2f3da7107"},"status":{"valid":true}}
         </data>
 <div id="js-fee-table-content">
 <div class="xvx-table-container" id="deposit">
 <p class="xvx-table-container__heading xvx-font-copy xvx-text-center"> Fee for Bank Deposit</p>
 <table class="xvx-table xvx-table--fee">
 <thead class="xvx-table--fee__head">
 <tr>
 <th class="xvx-table--fee__head-th xvx-font-copy-legal table-col-xs-8" scope="col"> Paying with</th>
 <th class="xvx-table--fee__head-th xvx-font-copy-legal table-col-xs-4" scope="col"> Fee in USD</th>
 </tr>
 </thead>
 <tbody class="xvx-table--fee__body">
 <tr class="xvx-table--fee__body-tr">
 <td class="

In [15]:
# Creating fee output for each page
for i in range(len(pages)): 
    fee = []
    page = pages[i]
    
    for j in delivery_type:
        ty = page.find_all('div',{'id':j})
    
        if ty == []:
            fee.append(0)
            fee.append(0)
            fee.append(0)
        else:
            for k in range(1,4):
                f = ty[0].find_all('td',{'class': 'xvx-table--fee__body-td xvx-font-copy fee-value'})[k].get_text()
                fee.append(float(f))
            
    df.insert(5+i,txn[i],fee,True)
    

In [16]:
df

,send_country,send_currency,receive_country,receive_currency,payment_type,50,100,200,300,400,...,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000
0,US,USD,MX,MXN,deposit Bank account,2.99,2.99,2.99,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,US,USD,MX,MXN,deposit Debit card,3.99,3.99,3.99,3.99,3.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,76.49
2,US,USD,MX,MXN,deposit Credit card,3.99,3.99,3.99,3.99,3.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,76.49
3,US,USD,MX,MXN,pickup Bank account,3.99,3.99,3.99,3.99,3.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,US,USD,MX,MXN,pickup Debit card,3.99,3.99,3.99,3.99,3.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,US,USD,MX,MXN,pickup Credit card,3.99,3.99,3.99,3.99,3.99,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,US,USD,MX,MXN,summarized Bank account,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,US,USD,MX,MXN,summarized Debit card,0.00,0.00,0.00,0.00,0.00,...,53.99,56.99,59.99,62.99,63.99,66.99,69.99,72.99,75.99,0.00
8,US,USD,MX,MXN,summarized Credit card,0.00,0.00,0.00,0.00,0.00,...,53.99,56.99,59.99,62.99,63.99,66.99,69.99,72.99,75.99,0.00


In [17]:
# Export to csv
today = datetime.today().strftime('%Y-%m-%d')

df.to_csv(str(today + sourceCountryCode + '_' + sourceCurrencyCode + '_' 
              + destinationCountryCode + '_' + destinationCurrencyCode + '_xoom_fee.csv'), index=False)